<a href="https://colab.research.google.com/github/WilliamAshbee/splineexample/blob/main/LSTM__wrheadx3_firstpartialsuccessameba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

With progressive growing of the sequence, the loss drops more stably. 

In [33]:
#TODO:TEST A DIFFERENT TYPE OF WRITE HEAD (ADD A SECOND LTM MEMORY MODULE)
#TODO:TEST A DIFFERENT TYPE OF LOSS TO SMOOTH OUT THE BLURRY READ/WRITE.

https://r2rt.com/non-zero-initial-states-for-recurrent-neural-networks.html

In [34]:
import torch
import numpy as np
import pylab as plt
from skimage import filters
import math

global numpoints
numpoints = 1000
side = 32

rows = torch.zeros(32,32)
columns = torch.zeros(32,32)


for i in range(32):
    columns[:,i] = i
    rows[i,:] = i


def donut_matrix(length = 10):

    radiusMax = side /3
    w = 1
    sigmas = [None, 1]
    
    canvas = torch.zeros((length,side, side))
    r0 = torch.tensor(np.random.uniform(side/4, side/3, length))

    radii = torch.zeros((length,numpoints))
    radii[:, :] = r0.unsqueeze(1)
    
    ind = [x for x in range(numpoints)]

    theta = torch.FloatTensor(ind)
    theta *= math.pi*2.0/(float)(numpoints)
    
    for i in range(1,length):
        a = np.random.uniform(1.0,3.0)*torch.sin(np.random.uniform(20.0)*theta+np.random.uniform(1000.0))
        #a = 4.0*torch.sin(10.0*theta)
        #print(a.shape,torch.max(a))
        radii[i,:] += a
        #print(radii.shape, torch.max(radii))
    
    assert torch.min(radii)>0
    #print(radii.max(axis = 0)[0].shape)
    rmaxs = radii.max(axis = 1)[0]
    pmins = rmaxs+1.0
    pmaxs = side-rmaxs-1.0
    x0 = np.random.uniform(pmins,pmaxs)
    y0 = np.random.uniform(pmins,pmaxs)
    x0[:]=side/2
    y0[:]=side/2
    x0 = torch.tensor(x0)
    y0 = torch.tensor(y0)
    
    x0 = x0.unsqueeze(1)
    y0 = y0.unsqueeze(1)
    #radii = torch.from_numpy(radii)
    xrfactors = torch.cos(theta).unsqueeze(0)
    yrfactors = torch.sin(theta).unsqueeze(0)
    
    print('x0_y0_r_xrf_yrf',x0.shape,y0.shape,radii.shape,xrfactors.shape,yrfactors.shape)

    x = (x0+(xrfactors*radii))
    y = (y0+(yrfactors*radii))
    assert x.shape == (length,numpoints)
    assert y.shape == (length,numpoints)
    assert torch.sum(x[x>(side-1)])==0 
    assert torch.sum(x[x<0])==0 
    assert torch.sum(y[y>(side-1)])==0 
    assert torch.sum(y[y<0])==0 
    
    points = torch.zeros(length,numpoints,2)
    for l in range(length):
        canvas[l,y[l,:].type(torch.LongTensor),x[l,:].type(torch.LongTensor)]=1.0
        points[l,:,0] = x[l,:]#modified for lstm discriminator
        points[l,:,1] = y[l,:]#modified for lstm discriminator 
    
    
    return {
        'canvas': canvas, 
        'points':points.type(torch.FloatTensor)}

def plot_all_model( sample = None, label=None,fig=None,ax=None):
    X = label[:,0]
    Y = label[:,1]
    img = sample[:,:].detach().squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    predres = 1000

    s = [.001 for x in range(predres)]
    
    assert len(s) == predres
    c = ['red' for x in range(predres)]
    assert len(c) == predres
    ax.scatter(X.detach().cpu().numpy(),Y.detach().cpu().numpy(),s = s,c = c)


def getXYs(model=None,loader_disp = None):

    outputs = torch.zeros(100,1000,2).cuda()
    h0 = None
    c0 = None
    
    for samples,labels in loader_disp:
      for i in range(38):
        optimizer.zero_grad()
        if i == 0:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i)
          outputs[:,i*25:(i+1)*25,:] = out.detach()
        else:
          out, h0, c0,o,output = model(samples.cuda(),labels[:,0,:2].cuda(),h0,c0,ind = i,o = o,output=output)
          outputs[:,i*25:(i+1)*25,:] = out.detach()

      return samples, outputs


          
        

def plot_all( sample = None, labels = None,fig = None,ax = None):
    img = sample[:,:].squeeze().cpu().numpy()
    ax.imshow(img, cmap=plt.cm.gray_r)
    X = labels[:,0]
    Y = labels[:,1]
    s = [.001 for x in range(numpoints)]
    c = ['red' for x in range(numpoints)]
    ax.scatter(X.cpu().numpy(),Y.cpu().numpy(),s = s,c = c)
    
class DonutDataset(torch.utils.data.Dataset):
    """Donut dataset."""
    def __init__(self, length = 10):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.length = length
        self.values = donut_matrix(length)
        assert self.values['canvas'].shape[0] == self.length
        assert self.values['points'].shape[0] == self.length
        
        count = 0
        for i in range(self.length):
          a = self[i]
          c = a[0]
          for el in a[1]:
            #print(c[(int)(el[1]),(int)(el[0])].item())
            #assert c[(int)(el[1]),(int)(el[0])].item() == 1
            y,x = (int)(el[1]),(int)(el[0])
            if x < side-2 and x > 2 and y < side-2 and y > 2: 
              if c[y,x] != 1 and \
                c[y+1,x] != 1 and c[y+1,-1+x] != 1 and c[y+1,1+x] != 1 and \
                c[y-1,x] != 1 and c[y,-1+x] != 1 and c[y,1+x] != 1:
                count+=1
        assert count ==0
    def __len__(self):
        return self.length


    def __getitem__(self, idx):
        canvas = self.values["canvas"]
        
        canvas = canvas[idx,:,:]
        assert canvas.shape == (side,side)
        
        points = self.values["points"]
        points = points[idx,:]
        #points = points.unsqueeze(1)
        #z = torch.zeros(numpoints,1)
        #print(z.shape)
        #points = torch.cat([points,z], dim = 1)
        
        #print('points', points.shape)
        return canvas, points
    
    @staticmethod
    def displayCanvas(title,ddataset, model):
        #model.setBatchSize(batch_size = 1)
        fig = plt.figure()
          
        if model != None:
          #model = model.eval()
          loader_disp = data.DataLoader(
            ddataset, 
            batch_size=100,
            num_workers=2)
          samples, outputs = getXYs(model=model,loader_disp = loader_disp)
          for i in range(100):
            ax = fig.add_subplot(10,10,i+1)
            plot_all_model(samples[i,:,:],outputs[i,:,:],fig=fig,ax=ax)
            plt.axis('off')

        else:
          for i in range(100):
            sample, labels = dataset[i]
            ax = fig.add_subplot(10,10,i+1)
            plot_all(sample = sample,labels = labels,fig = fig, ax= ax)
            plt.axis('off')

        fig.savefig(title,dpi=450)
        plt.close(fig) #where f is the figure
        plt.clf()
        plt.cla()
        plt.close()#should free memory
        print('open file: ',title)


In [35]:
dataset = DonutDataset(length = 100)

DonutDataset.displayCanvas('donut.png',dataset, model = None)


x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])
open file:  donut.png


In [36]:

from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

mini_batch = 100
dataset = DonutDataset(length = 100*20)
loader_train = data.DataLoader(
    dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([2000, 1]) torch.Size([2000, 1]) torch.Size([2000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [37]:
mini_batch = 100
test_dataset = DonutDataset(length = 1000)
loader_test = data.DataLoader(
    test_dataset, 
    batch_size=mini_batch,
    sampler=RandomSampler(data_source=test_dataset),
    num_workers=2)

x0_y0_r_xrf_yrf torch.Size([1000, 1]) torch.Size([1000, 1]) torch.Size([1000, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [38]:
pred_dataset = DonutDataset(length = 100)

x0_y0_r_xrf_yrf torch.Size([100, 1]) torch.Size([100, 1]) torch.Size([100, 1000]) torch.Size([1, 1000]) torch.Size([1, 1000])


In [71]:
import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url

class DecoderRNN(nn.Module):
    def __init__(self):
        super(DecoderRNN, self).__init__()
        self.track = 0
        # define the properties
        self.embed_size = 2
        self.hidden_size = 256
        self.num_layers = 2
        self.seq_len = 25
        self.full_len = 1000
        self.longmem_len = int(self.full_len/self.seq_len)
        self.lstm_cell = nn.LSTM(self.hidden_size*2, self.hidden_size,self.num_layers)

        self.inject_image =  nn.Sequential(
            nn.Linear(2*1000+1024+8*self.hidden_size,self.hidden_size*2),
            nn.ReLU()
        )

        
        self.fc_out = nn.Sequential(
            nn.Linear(2+self.hidden_size*10,10000),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(10000,2),
            nn.Sigmoid()
        )
        self.hi_1 = nn.Sequential(
            nn.Linear(2+2*1000+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )
        self.hi_2 = nn.Sequential(
            nn.Linear(2+2*1000+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )

        self.ci_1 = nn.Sequential(
            nn.Linear(2+2*1000+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )
        self.ci_2 = nn.Sequential(
            nn.Linear(2+2*1000+1024+self.hidden_size*2,self.hidden_size),
            nn.Tanh()
        )

        self.alpha_1 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.seq_len),
            nn.Softmax()
        )
        
        self.alpha_long1 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long2 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long3 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.alpha_long4 = nn.Sequential(
            nn.Linear(self.hidden_size*2,self.longmem_len),
            nn.Softmax()
        )
        
        self.long_term_write_1 = nn.Sequential(
            nn.Linear(self.hidden_size*(self.seq_len+1),self.hidden_size),
            nn.ReLU()
        )
        
        self.long_term_write_2 = nn.Sequential(
            nn.Linear(self.hidden_size*(self.seq_len+1),self.hidden_size),
            nn.ReLU()
        )
        

        self.embs = None #= torch.empty((self.batch_size, 1000, self.hidden_size)).cuda()
        
        self.ltm_temp_1 = None
        self.ltm_temp_2 = None
        self.longtermMem_1 = None
        self.longtermMem_2 = None
        self.longtermMemP = None
                

            
        
    def forward(self, features, p0,h0 = None, c0= None, ind = 0, o = None,output = None):
        #print('a')
        assert features != None
        self.track +=1
        batch_size = features.size(0)
        
        if o == None:
          out = p0
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          self.longtermMemP = torch.zeros(batch_size,1000,2).cuda()
          self.longtermMem_1 = torch.zeros(batch_size,self.longmem_len,self.hidden_size).cuda()
          self.longtermMem_2 = torch.zeros(batch_size,self.longmem_len,self.hidden_size).cuda()
          self.ltm_temp_1 = torch.zeros(batch_size,self.hidden_size).cuda()
          self.ltm_temp_2 = torch.zeros(batch_size,self.hidden_size).cuda()
        else:
          self.ltm_temp_1 = self.long_term_write_1(torch.cat([torch.flatten(self.embs,start_dim=1),output.squeeze()],dim=1))
          self.ltm_temp_2 = self.long_term_write_1(torch.cat([torch.flatten(self.embs,start_dim=1),output.squeeze()],dim=1))
          self.longtermMem_1[:, ind-1, :] = self.ltm_temp_1
          self.longtermMem_2[:, ind-1, :] = self.ltm_temp_2
          self.embs = torch.empty((batch_size, self.seq_len, self.hidden_size)).cuda()
          out = o

        #print('c')
        if h0 == None and c0 == None:
          combin_init = torch.cat([torch.flatten(self.longtermMemP,start_dim=1),torch.flatten(features,start_dim=1),out,torch.zeros(batch_size,self.hidden_size).cuda(),torch.zeros(batch_size,self.hidden_size).cuda()],dim=1)
        
          
          c0 = torch.zeros(self.num_layers,batch_size,self.hidden_size).cuda()
          h_0_1 = self.hi_1(combin_init)
          h_0_2 = self.hi_2(combin_init)
          #c_0_1 = self.ci_1(combin_init)
          #c_0_2 = self.ci_2(combin_init)
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          #c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        else:
          combin_init = torch.cat([torch.flatten(self.longtermMemP,start_dim=1),torch.flatten(features,start_dim=1),out,output.squeeze(),h0[0,:,:]],dim=1)
          h_0_1 = self.hi_1(combin_init)
          
          combin_init = torch.cat([torch.flatten(self.longtermMemP,start_dim=1),torch.flatten(features,start_dim=1),out,output.squeeze(),h0[1,:,:]],dim=1)
          h_0_2 = self.hi_2(combin_init)
          
          #combin_init = torch.cat([torch.flatten(self.longtermMemP,start_dim=1),torch.flatten(features,start_dim=1),out,output.squeeze(),c0[0,:,:]],dim=1)
          #c_0_1 = self.ci_1(combin_init)
          
          #combin_init = torch.cat([torch.flatten(self.longtermMemP,start_dim=1),torch.flatten(features,start_dim=1),out,output.squeeze(),c0[1,:,:]],dim=1)
          #c_0_2 = self.ci_2(combin_init)
          
          h0 = torch.stack([h_0_1,h_0_2],dim = 0)
          #c0 = torch.stack([c_0_1,c_0_2],dim = 0)
        
        
        hidden_state = h0
        cell_state = c0
        outputs = torch.empty((batch_size, self.seq_len, 2)).cuda()
        
        output = None
        #print('b')
        maxhidden = torch.zeros(self.seq_len)
        minhidden = torch.zeros(self.seq_len)
        mincell = torch.zeros(self.seq_len)
        maxcell = torch.zeros(self.seq_len)
        maxout = torch.zeros(self.seq_len)
        minout = torch.zeros(self.seq_len)
        for t in range(self.seq_len):
        
            # for the first time step the input is the feature vector
            if output == None:
              output = torch.zeros(batch_size,self.hidden_size).cuda()
              outesread1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl1 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl2 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl3 = torch.zeros(batch_size,self.hidden_size).cuda()
              outesreadl4 = torch.zeros(batch_size,self.hidden_size).cuda()
            
            combin_lstm = torch.cat([torch.flatten(features,start_dim=1),torch.flatten(self.longtermMemP,start_dim=1),output.squeeze(),outesread1,outesreadl1,outesreadl2,outesreadl3,outesreadl4,self.ltm_temp_1,self.ltm_temp_2],dim=1)
            combin_lstm = self.inject_image(combin_lstm)
            

            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            
            output, (hidden_state, cell_state) = self.lstm_cell(combin_lstm.unsqueeze(0), (hidden_state, cell_state))
            #print(hidden_state.shape,cell_state.shape)
            maxhidden[t] = torch.max(hidden_state)
            minhidden[t] = torch.min(hidden_state)
            maxcell[t] = torch.max(cell_state)
            mincell[t] = torch.min(cell_state)
            maxout[t] = torch.max(output)
            minout[t] = torch.min(output)
            
            outputs[:, t, :] = out
            self.longtermMemP[:,self.seq_len*ind+t,:] = out.detach()
            
            self.embs[:, t, :] = output.squeeze().clone().detach()
            

            a1 = self.alpha_1(combin_lstm)
            outesreada1 = self.embs.clone()*a1.unsqueeze(2)#clone
            outesread1 = torch.sum(outesreada1,dim=1)
            
            
            #print('d')
            al1 = self.alpha_long1(combin_lstm)
            outesreadal1 = self.longtermMem_1.detach()*al1.unsqueeze(2)#clone
            outesreadl1 = torch.sum(outesreadal1,dim=1)
            
            al2 = self.alpha_long2(combin_lstm)
            outesreadal2 = self.longtermMem_1.detach()*al2.unsqueeze(2)#clone
            outesreadl2 = torch.sum(outesreadal2,dim=1)
            
            al3 = self.alpha_long3(combin_lstm)
            outesreadal3 = self.longtermMem_2.detach()*al3.unsqueeze(2)#clone
            outesreadl3 = torch.sum(outesreadal3,dim=1)
            
            al4 = self.alpha_long4(combin_lstm)
            outesreadal4 = self.longtermMem_2.detach()*al4.unsqueeze(2)#clone
            outesreadl4 = torch.sum(outesreadal4,dim=1)
            

            combOut = torch.cat([combin_lstm, output.squeeze(),out,outesread1,outesreadl1,outesreadl2,outesreadl3,outesreadl4,self.ltm_temp_1,self.ltm_temp_2],dim=1)#,outesread],dim=1)
            

            out = self.fc_out(combOut)
            out = out*32.0
            # build the output tensor
            
        
        print(torch.max(maxhidden),torch.min(minhidden),torch.max(maxcell),torch.min(mincell),torch.max(maxout),torch.min(minout))    
        #self.embs = self.embs.detach().clone()
        return outputs, hidden_state.detach(), cell_state.detach(), out.detach(),output.detach()#, captions

In [72]:
model = DecoderRNN().cuda()

In [73]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal


In [74]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [75]:
for seql in range(3,23,5):
  #seql = 19
  for epoch in range (5):
    loss = None
    #model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Streaming output truncated to the last 5000 lines.
tensor(0.7590, grad_fn=<MaxBackward1>) tensor(-0.8972, grad_fn=<MinBackward1>) tensor(2.6456, grad_fn=<MaxBackward1>) tensor(-3.1652, grad_fn=<MinBackward1>) tensor(0.4990, grad_fn=<MaxBackward1>) tensor(-0.5071, grad_fn=<MinBackward1>)
tensor(0.6731, grad_fn=<MaxBackward1>) tensor(-0.8773, grad_fn=<MinBackward1>) tensor(2.5577, grad_fn=<MaxBackward1>) tensor(-3.4600, grad_fn=<MinBackward1>) tensor(0.4647, grad_fn=<MaxBackward1>) tensor(-0.5007, grad_fn=<MinBackward1>)
tensor(0.6939, grad_fn=<MaxBackward1>) tensor(-0.8358, grad_fn=<MinBackward1>) tensor(2.2715, grad_fn=<MaxBackward1>) tensor(-2.6955, grad_fn=<MinBackward1>) tensor(0.4039, grad_fn=<MaxBackward1>) tensor(-0.4911, grad_fn=<MinBackward1>)
epoch 0 seql 13
train loss tensor(1.6159, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(0.5718, grad_fn=<MaxBackward1>) tensor(-0.5740, grad_fn=<MinBackward1>) tensor(1.3903, grad_fn=<MaxBackward1>) tensor(-1.5728, grad_fn=<MinBackward

In [76]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate1.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(0.6523) tensor(-0.7390) tensor(3.6048) tensor(-4.4858) tensor(0.5031) tensor(-0.5708)
tensor(0.8451) tensor(-0.7099) tensor(2.9689) tensor(-7.3368) tensor(0.3641) tensor(-0.4568)
tensor(0.9264) tensor(-0.8129) tensor(7.4718) tensor(-10.1298) tensor(0.3478) tensor(-0.4608)
tensor(0.9436) tensor(-0.8139) tensor(8.8890) tensor(-11.0151) tensor(0.3473) tensor(-0.4593)
tensor(0.9417) tensor(-0.7804) tensor(8.4623) tensor(-9.7181) tensor(0.3063) tensor(-0.4192)
tensor(0.8930) tensor(-0.7678) tensor(6.7285) tensor(-6.0974) tensor(0.3658) tensor(-0.4489)
tensor(0.8571) tensor(-0.7548) tensor(6.9271) tensor(-5.6022) tensor(0.3348) tensor(-0.4462)
tensor(0.8637) tensor(-0.8359) tensor(6.9196) tensor(-5.0865) tensor(0.3193) tensor(-0.4126)
tensor(0.9289) tensor(-0.8461) tensor(7.4344) tensor(-9.4251) tensor(0.3131) tensor(-0.3678)
tensor(0.9054) tensor(-0.8608) tensor(11.2512) tensor(-19.5481) tensor(0.3283) tensor(-0.3887)
tensor(0.9056) tensor(-0.8791) tensor(10.8881) tensor(-22.8971) te

In [65]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001, betas = (.9,.999))#ideal

for g in optimizer.param_groups:
    g['lr'] = 0.0001


In [77]:
#for seql in range(15,20,4):
seql = 23
for epoch in range (20):
  #model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    #model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Streaming output truncated to the last 5000 lines.
tensor(0.8340, grad_fn=<MaxBackward1>) tensor(-0.8798, grad_fn=<MinBackward1>) tensor(14.0224, grad_fn=<MaxBackward1>) tensor(-5.7130, grad_fn=<MinBackward1>) tensor(0.6189, grad_fn=<MaxBackward1>) tensor(-0.6258, grad_fn=<MinBackward1>)
tensor(0.8683, grad_fn=<MaxBackward1>) tensor(-0.8834, grad_fn=<MinBackward1>) tensor(14.8773, grad_fn=<MaxBackward1>) tensor(-22.1804, grad_fn=<MinBackward1>) tensor(0.3231, grad_fn=<MaxBackward1>) tensor(-0.4509, grad_fn=<MinBackward1>)
tensor(0.8723, grad_fn=<MaxBackward1>) tensor(-0.9042, grad_fn=<MinBackward1>) tensor(15.0280, grad_fn=<MaxBackward1>) tensor(-38.8493, grad_fn=<MinBackward1>) tensor(0.2744, grad_fn=<MaxBackward1>) tensor(-0.4549, grad_fn=<MinBackward1>)
tensor(0.8422, grad_fn=<MaxBackward1>) tensor(-0.8783, grad_fn=<MinBackward1>) tensor(9.5025, grad_fn=<MaxBackward1>) tensor(-56.4191, grad_fn=<MinBackward1>) tensor(0.3087, grad_fn=<MaxBackward1>) tensor(-0.6114, grad_fn=<MinBackwar

In [67]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate2.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(0.9902) tensor(-0.9673) tensor(11.7231) tensor(-11.5154) tensor(0.2325) tensor(-0.1819)
tensor(0.9819) tensor(-0.9676) tensor(15.8330) tensor(-14.0453) tensor(0.0804) tensor(-0.1679)
tensor(0.9694) tensor(-0.9481) tensor(17.1628) tensor(-11.7854) tensor(0.0744) tensor(-0.1704)
tensor(0.9390) tensor(-0.9488) tensor(14.9115) tensor(-9.7555) tensor(0.0532) tensor(-0.1645)
tensor(0.9231) tensor(-0.9495) tensor(12.6857) tensor(-7.2899) tensor(0.1016) tensor(-0.1701)
tensor(0.9295) tensor(-0.9484) tensor(11.0306) tensor(-7.1810) tensor(0.1753) tensor(-0.2662)
tensor(0.9175) tensor(-0.9439) tensor(12.9735) tensor(-5.9264) tensor(0.1650) tensor(-0.2385)
tensor(0.9548) tensor(-0.9754) tensor(9.3072) tensor(-9.1314) tensor(0.2609) tensor(-0.3393)
tensor(0.9697) tensor(-0.9518) tensor(9.1638) tensor(-11.0590) tensor(0.2912) tensor(-0.3911)
tensor(0.9823) tensor(-0.9523) tensor(15.6816) tensor(-13.7196) tensor(0.2656) tensor(-0.3700)
tensor(0.9891) tensor(-0.9596) tensor(19.7831) tensor(-18

In [78]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [79]:

#model = model.train()
#for seql in range(15,20,4):
seql = 23
for epoch in range (10):
  loss = None
  #model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Streaming output truncated to the last 5000 lines.
tensor(0.9465) tensor(-0.9117) tensor(17.2035) tensor(-119.1602) tensor(0.3578) tensor(-0.8321)
tensor(0.9174) tensor(-0.9361) tensor(9.7223) tensor(-128.5219) tensor(0.2930) tensor(-0.7941)
tensor(0.8556) tensor(-0.9888) tensor(14.1798) tensor(-143.6620) tensor(0.4048) tensor(-0.8357)
tensor(0.8901) tensor(-0.9919) tensor(23.6742) tensor(-161.6965) tensor(0.4540) tensor(-0.8270)
tensor(0.8838) tensor(-0.9928) tensor(32.6858) tensor(-180.2711) tensor(0.4431) tensor(-0.8353)
tensor(0.8975) tensor(-0.9911) tensor(33.7863) tensor(-192.3905) tensor(0.4139) tensor(-0.8059)
tensor(0.9671) tensor(-0.9820) tensor(31.6856) tensor(-193.2627) tensor(0.4740) tensor(-0.8221)
tensor(0.9836) tensor(-0.9871) tensor(38.1308) tensor(-171.7604) tensor(0.5358) tensor(-0.8023)
tensor(0.9898) tensor(-0.9825) tensor(52.6466) tensor(-147.0549) tensor(0.5700) tensor(-0.8265)
tensor(0.9914) tensor(-0.9906) tensor(62.0444) tensor(-135.0212) tensor(0.5465) tensor

KeyboardInterrupt: ignored

ran so well, interrupted

In [80]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate3.png',pred_dataset, model = model)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor(0.8737) tensor(-0.9174) tensor(12.0386) tensor(-5.2388) tensor(0.6159) tensor(-0.6518)
tensor(0.9124) tensor(-0.9130) tensor(11.4345) tensor(-11.3900) tensor(0.4562) tensor(-0.6712)
tensor(0.8945) tensor(-0.8851) tensor(10.3587) tensor(-29.7388) tensor(0.3219) tensor(-0.5958)
tensor(0.6224) tensor(-0.9023) tensor(8.1524) tensor(-48.5955) tensor(0.3930) tensor(-0.6870)
tensor(0.7074) tensor(-0.9595) tensor(14.8537) tensor(-68.2177) tensor(0.3796) tensor(-0.7335)
tensor(0.8947) tensor(-0.8264) tensor(15.9334) tensor(-86.1167) tensor(0.3885) tensor(-0.8136)
tensor(0.8254) tensor(-0.8507) tensor(14.5562) tensor(-96.6204) tensor(0.4182) tensor(-0.8507)
tensor(0.8934) tensor(-0.8364) tensor(7.9791) tensor(-99.2735) tensor(0.4078) tensor(-0.8322)
tensor(0.8908) tensor(-0.9690) tensor(9.2989) tensor(-103.2536) tensor(0.3761) tensor(-0.7911)
tensor(0.9196) tensor(-0.9904) tensor(13.3052) tensor(-115.2019) tensor(0.4114) tensor(-0.8298)
tensor(0.9098) tensor(-0.9930) tensor(15.9195) tenso

In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [ ]:
for seql in range(23,39,4):
  #seql = 19
  for epoch in range (5):
    loss = None
    #model = model.train()

    for x,y in loader_train:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        optimizer.zero_grad()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        #writer.add_scalar("Loss/train", loss, epoch)
        
        loss.backward()
        optimizer.step()
      #print(y[:,0,:2])
      print('epoch',epoch,'seql',seql)
      print('train loss', loss)
    
    with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

In [ ]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate4.png',pred_dataset, model = model)

In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.0001



In [ ]:
#for seql in range(15,20,4):
seql = 38
for epoch in range (20):
  #model = model.train()
  loss = None
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
    #model = model.eval()
    for x,y in loader_test:
      h0 = None
      c0 = None
      for i in range(seql):
        x = x.cuda()
        y = y.cuda()
        
        if i == 0:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
        else:
          out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
        
        loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      
        assert not torch.any(torch.isnan(out)).item()
        assert not torch.any(torch.isnan(y)).item()
      
      print('epoch',epoch,'seql',seql)
      print('testloss',loss)


In [ ]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('intermediate5.png',pred_dataset, model = model)

In [ ]:
#optimizer = torch.optim.Adam(model.parameters(),lr = 0.00001, betas = (.9,.999))#ideal
for g in optimizer.param_groups:
    g['lr'] = 0.00001



In [ ]:

#model = model.train()
#for seql in range(15,20,4):
seql = 38
for epoch in range (60):
  loss = None
  #model = model.train()
  for x,y in loader_train:
    h0 = None
    c0 = None
    for i in range(seql):
      x = x.cuda()
      y = y.cuda()
      
      optimizer.zero_grad()
      
      if i == 0:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
      else:
        out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
      loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
      writer.add_scalar("Loss/train", loss, epoch)
      
      loss.backward()
      optimizer.step()
    #print(y[:,0,:2])
    print('epoch',epoch,'seql',seql)
    print(loss)
  with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        

In [ ]:
writer.flush()
writer.close()


In [ ]:
#%load_ext tensorboard

In [ ]:
#!kill 3285
#%tensorboard --logdir runs

In [ ]:
#!nvidia-smi

In [ ]:
with torch.no_grad():
  #model = model.eval()
  DonutDataset.displayCanvas('lstmpredictions.png',pred_dataset, model = model)

In [ ]:
for name, param in model.named_parameters():
  print(name)

In [ ]:
with torch.no_grad():
      #model = model.eval()
      for x,y in loader_test:
        h0 = None
        c0 = None
        for i in range(seql):
          x = x.cuda()
          y = y.cuda()
          
          if i == 0:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i)
          else:
            out, h0, c0,o,output = model(x,y[:,0,:2],h0,c0,ind = i,o = o,output=output)
          
          loss = torch.mean((out-y[:,1+(25*i):1+(25*(i+1)),:2])**2)
        
          assert not torch.any(torch.isnan(out)).item()
          assert not torch.any(torch.isnan(y)).item()
        
        print('epoch',epoch,'seql',seql)
        print('testloss',loss)
  
        